In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import collections
from itertools import combinations

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
display(data.sample(5))
data.describe()

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
830,tt1578275,70000000,67112664,The Dilemma,Kevin James|Vince Vaughn|Winona Ryder|Jennifer...,Ron Howard,Two best friends. Nothing could come between t...,Longtime friends Ronny and Nick are partners i...,111,Comedy|Drama,Imagine Entertainment|Universal Pictures|Spygl...,1/13/2011,5.2,2011
656,tt0430922,28000000,92380927,Role Models,Seann William Scott|Paul Rudd|Elizabeth Banks|...,David Wain,They're about to get more than they plea-barga...,Two salesmen trash a company truck on an energ...,99,Comedy,Universal Pictures|Internationale Filmprodukti...,2/3/2008,6.3,2008
202,tt2106361,50000000,160602194,Into the Storm,Richard Armitage|Sarah Wayne Callies|Matt Wals...,Steven Quale,Prepare to go,The town of Silverton is in one day destroyed ...,89,Action|Thriller,New Line Cinema|Village Roadshow Pictures|Brok...,8/6/2014,5.7,2014
1316,tt0401792,40000000,158733820,Sin City,Jessica Alba|Devon Aoki|Alexis Bledel|Powers B...,Robert Rodriguez|Frank Miller|Quentin Tarantino,Walk down the right back alley in Sin City and...,Welcome to Sin City. This town beckons to the ...,124,Action|Thriller|Crime,Dimension Films|Troublemaker Studios|Miramax,3/31/2005,7.1,2005
1440,tt0317919,150000000,397850012,Mission: Impossible III,Tom Cruise|Philip Seymour Hoffman|Ving Rhames|...,J.J. Abrams,The Mission Begins 05:05:06.,Retired from active duty to train new IMF agen...,126,Adventure|Action|Thriller,Paramount Pictures|Cruise/Wagner Productions|S...,5/3/2006,6.3,2006


,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [3]:
answers = {} # создадим словарь для ответов

In [4]:
# Создаем переменную profit, равную прибыльности фильма 
profit = data.revenue - data.budget
# Вычислим среднее значение бюджетов фильмов
budget_mean = data.budget.mean()
# В основной Датафрейм добавляем колонку со значением прибыльности фильма
data['profit'] = profit
# В основной Датафрейм добавляем колонку со списком жанров
data['genres_list'] = data['genres'].apply(lambda x: x.split('|'))
# В основной Датафрейм добавляем колонку со списком режиссеров, снявших данный фильм
data['director_list'] = data['director'].apply(lambda x: str(x).split('|'))
# В основной Датафрейм добавляем колонку со списком актеров
data['cast_list'] = data['cast'].apply(lambda x: x.split('|'))
# В основной Датафрейм добавляем колонку со списком компаний, выпустивших данный фильм
data['production_companies_list'] = data['production_companies'].apply(lambda x: x.split('|'))
# В основной Датафрейм добавляем колонку с количеством символов в названии фильма
data['lenght_title'] = data['original_title'].apply(lambda x: len(x))
# В основной Датафрейм добавляем колонку с количеством слов в описании фильма
data['lenght_overview'] = data['overview'].apply(lambda x: x.split()).apply(len)

In [5]:
# Создаем Датафрейм с фильмами, вышедшими только в 2008 году
data_2008 = data[(data.release_year == 2008)]
# Создаем Датафрейм с фильмами, вышедшими в период с 2012 по 2014 год включительно
data_2012_14 = data[(data.release_year >= 2012) & (data.release_year <= 2014)]
# Создаем Датафрейм с прибыльными фильмами
data_pr = data[data.profit > 0]
# Создаем Датафрейм с фильмам в жанре Action
data_act = data[data.genres.str.contains('Action')]
# Создаем Датафрейм с фильмами, вышедшими только в 2012 году и добавляем в него колонку со списком актеров
data_2012 = data[data.release_year == 2012].copy()
data_2012['cast_list'] = data_2012['cast'].apply(lambda x: str(x).split('|'))
# Создаем временный Датафрейм, в который добавляем колонку с месяцем производства фильма
data_time = data.copy()
data_time['release_date_list'] = data_time['release_date'].apply(lambda x: x.split("/"))
data_time['release_month'] = data_time['release_date_list'].apply(lambda x: x[0])
# Создаем временный Датафрейм и в нем добавляем колонку со списком актеров
data_p = data.copy()
data_p['pairs'] = data_p['cast_list'].apply(lambda s: list(combinations(s, 2)))

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

ВАРИАНТ 1

In [6]:
# если необходтимо вывести целую строку Датафрейма:
data[data.budget == data.budget.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,genres_list,director_list,cast_list,production_companies_list,lenght_title,lenght_overview
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...,Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,Adventure|Action|Fantasy,...,5/11/2011,6.3,2011,641683000,"[Adventure, Action, Fantasy]",[Rob Marshall],"[Johnny Depp, PenÃ©lope Cruz, Geoffrey Rush, I...","[Walt Disney Pictures, Jerry Bruckheimer Films...",43,75


ВАРИАНТ 2

In [7]:
# если необходимо узнать только название фильма, его imdb_id и размер бюджета:
data.loc[data.budget == data.budget.max()][['imdb_id', 'original_title', 'budget']]

,imdb_id,original_title,budget
723,tt1298650,Pirates of the Caribbean: On Stranger Tides,380000000


In [8]:
answers['1'] = '723. Pirates of the Caribbean: On Stranger Tides (tt1298650)' # +

# 2. Какой из фильмов самый длительный (в минутах)?

ВАРИАНТ 1

In [9]:
# если необходтимо вывести целую строку Датафрейма:
data[data.runtime == data.runtime.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,genres_list,director_list,cast_list,production_companies_list,lenght_title,lenght_overview
1157,tt0279111,56000000,12923936,Gods and Generals,Stephen Lang|Jeff Daniels|Robert Duvall|Kevin ...,Ronald F. Maxwell,The nations heart was touched by...,The film centers mostly around the personal an...,214,Drama|History|War,...,2/21/2003,5.8,2003,-43076064,"[Drama, History, War]",[Ronald F. Maxwell],"[Stephen Lang, Jeff Daniels, Robert Duvall, Ke...","[Turner Pictures, Antietam Filmworks]",17,48


ВАРИАНТ 2

In [10]:
# если необходимо узнать только название фильма, его imdb_id и его длительность:
data.loc[data.runtime == data.runtime.max()][['imdb_id', 'original_title', 'runtime']]

,imdb_id,original_title,runtime
1157,tt0279111,Gods and Generals,214


In [11]:
answers['2'] = '1157. Gods and Generals (tt0279111)' # +

# 3. Какой из фильмов самый короткий (в минутах)?





ВАРИАНТ 1

In [12]:
# если необходтимо вывести целую строку Датафрейма:
data[data.runtime == data.runtime.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,genres_list,director_list,cast_list,production_companies_list,lenght_title,lenght_overview
768,tt1449283,30000000,14460000,Winnie the Pooh,Jim Cummings|Travis Oates|Jim Cummings|Bud Luc...,Stephen Anderson|Don Hall,Oh Pooh.,"During an ordinary day in Hundred Acre Wood, W...",63,Animation|Family,...,4/13/2011,6.8,2011,-15540000,"[Animation, Family]","[Stephen Anderson, Don Hall]","[Jim Cummings, Travis Oates, Jim Cummings, Bud...","[Walt Disney Pictures, Walt Disney Animation S...",15,52


ВАРИАНТ 2

In [13]:
# если необходимо узнать только название фильма, его imdb_id и его длительность:
data.loc[data.runtime == data.runtime.min()][['imdb_id', 'original_title', 'runtime']]

,imdb_id,original_title,runtime
768,tt1449283,Winnie the Pooh,63


In [14]:
answers['3'] = '768. Winnie the Pooh (tt1449283)' #+

# 4. Какова средняя длительность фильмов?


In [15]:
round(data.runtime.mean())

110

In [16]:
answers['4'] = '110' # +

# 5. Каково медианное значение длительности фильмов? 

In [17]:
round(data.runtime.median()) # +

107

In [18]:
answers['5'] = '107.0'

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

ВАРИАНТ 1

In [19]:
# если необходтимо вывести целую строку Датафрейма:
data[data.profit == data.profit.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,genres_list,director_list,cast_list,production_companies_list,lenght_title,lenght_overview
239,tt0499549,237000000,2781505847,Avatar,Sam Worthington|Zoe Saldana|Sigourney Weaver|S...,James Cameron,Enter the World of Pandora.,"In the 22nd century, a paraplegic Marine is di...",162,Action|Adventure|Fantasy|Science Fiction,...,12/10/2009,7.1,2009,2544505847,"[Action, Adventure, Fantasy, Science Fiction]",[James Cameron],"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[Ingenious Film Partners, Twentieth Century Fo...",6,28


ВАРИАНТ 2

In [20]:
# если необходимо узнать только название фильма, его imdb_id и размер прибыли:
data.loc[data.profit == data.profit.max()][['imdb_id', 'original_title', 'profit']]

,imdb_id,original_title,profit
239,tt0499549,Avatar,2544505847


In [21]:
answers['6'] = '239. Avatar (tt0499549)' # +

# 7. Какой фильм самый убыточный? 

ВАРИАНТ 1

In [22]:
# если необходтимо вывести целую строку Датафрейма:
data[data.profit == data.profit.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,genres_list,director_list,cast_list,production_companies_list,lenght_title,lenght_overview
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,...,7/3/2013,6.0,2013,-165710090,"[Action, Adventure, Western]",[Gore Verbinski],"[Johnny Depp, Armie Hammer, William Fichtner, ...","[Walt Disney Pictures, Jerry Bruckheimer Films...",15,68


ВАРИАНТ 2

In [23]:
# если необходимо узнать только название фильма, его imdb_id и размер прибыли:
data.loc[data.profit == data.profit.min()][['imdb_id', 'original_title', 'profit']]

,imdb_id,original_title,profit
1245,tt1210819,The Lone Ranger,-165710090


In [24]:
answers['7'] = '1245. The Lone Ranger (tt1210819)' # +

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [25]:
data[data.revenue > data.budget].imdb_id.count()

1478

In [26]:
answers['8'] = '1478' # +

# 9. Какой фильм оказался самым кассовым в 2008 году?

ВАРИАНТ 1

In [27]:
# если необходтимо вывести целую строку Датафрейма:
data_2008[data_2008.revenue == data_2008.revenue.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,genres_list,director_list,cast_list,production_companies_list,lenght_title,lenght_overview
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,...,7/16/2008,8.1,2008,816921825,"[Drama, Action, Crime, Thriller]",[Christopher Nolan],"[Christian Bale, Michael Caine, Heath Ledger, ...","[DC Comics, Legendary Pictures, Warner Bros., ...",15,67


ВАРИАНТ 2

In [28]:
# если необходимо узнать только название фильма, его imdb_id и величину кассовых сборов:
data_2008.loc[data_2008.revenue == data_2008.revenue.max()][['imdb_id', 'original_title', 'revenue']]

,imdb_id,original_title,revenue
599,tt0468569,The Dark Knight,1001921825


In [29]:
answers['9'] = '599. The Dark Knight (tt0468569)' # +

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


ВАРИАНТ 1

In [30]:
# если необходтимо вывести целую строку Датафрейма:
data_2012_14[data_2012_14.profit == data_2012_14.profit.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,genres_list,director_list,cast_list,production_companies_list,lenght_title,lenght_overview
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,...,7/3/2013,6.0,2013,-165710090,"[Action, Adventure, Western]",[Gore Verbinski],"[Johnny Depp, Armie Hammer, William Fichtner, ...","[Walt Disney Pictures, Jerry Bruckheimer Films...",15,68


ВАРИАНТ 2

In [31]:
# если необходимо узнать только название фильма, его imdb_id и размер прибыли:
data_2012_14.loc[data_2012_14.profit == data_2012_14.profit.min()][['imdb_id', 'original_title', 'profit']]

,imdb_id,original_title,profit
1245,tt1210819,The Lone Ranger,-165710090


In [32]:
answers['10'] = '1245. The Lone Ranger (tt1210819)' # +

# 11. Какого жанра фильмов больше всего?

In [33]:
c = collections.Counter()
for string in data['genres_list']:
    for name in string:
        c[name] +=1
c.most_common(1)

[('Drama', 782)]

In [34]:
answers['11'] = 'Drama' # +

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [35]:
c = collections.Counter()
for string in data_pr['genres_list']:
    for name in string:
        c[name] +=1
c.most_common(1)

[('Drama', 560)]

In [36]:
answers['12'] = 'Drama' # +

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [37]:
data_dir = data.explode('director_list')
data_temp = data_dir.groupby(['director_list'])[['revenue']].sum().sort_values('revenue', ascending=False)
data_temp.head(1)

,revenue
director_list,
Peter Jackson,6490593685


In [38]:
answers['13'] = 'Peter Jackson' # +

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [39]:
c = collections.Counter()
for string in data_act['director_list']:
    for name in string:
        c[name] +=1
c.most_common(1)

[('Robert Rodriguez', 9)]

In [40]:
answers['14'] = 'Robert Rodriguez' # +

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [41]:
data_cast_2012 = data_2012.explode('cast_list')
data_cast_2012.groupby(['cast_list'])[['revenue']].sum().sort_values('revenue', ascending=False).head(1)

,revenue
cast_list,
Chris Hemsworth,2027450773


In [42]:
answers['15'] = 'Chris Hemsworth' # +

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [43]:
data_temp2 = data.explode('cast_list')
data_cast = data_temp2[data_temp2.budget > budget_mean].cast_list
data_cast.value_counts().head(1)

Matt Damon    18
Name: cast_list, dtype: int64

In [44]:
answers['16'] = 'Matt Damon' # +

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [45]:
# Создаем Датафрейм с фильмами, в которых снимался Nicolas Cage
data_cage = data_temp2[data_temp2.cast_list == 'Nicolas Cage']
data_cage_temp = data_cage.explode('genres_list')
data_cage_temp['genres_list'].value_counts().head(1)

Action    17
Name: genres_list, dtype: int64

In [46]:
answers['17'] = 'Action' # +

# 18. Самый убыточный фильм от Paramount Pictures

In [47]:
data_prod = data.explode('production_companies_list')
data_prod.loc[data_prod['production_companies_list'].str.contains("Paramount")].sort_values('profit').head(1)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,genres_list,director_list,cast_list,production_companies_list,lenght_title,lenght_overview
925,tt0267626,100000000,35168966,K-19: The Widowmaker,Harrison Ford|Liam Neeson|Peter Sarsgaard|Joss...,Kathryn Bigelow,Fate has found its hero.,When Russia's first nuclear submarine malfunct...,138,Thriller|Drama|History,...,7/19/2002,6.0,2002,-64831034,"[Thriller, Drama, History]",[Kathryn Bigelow],"[Harrison Ford, Liam Neeson, Peter Sarsgaard, ...",Paramount Pictures,20,23


In [48]:
answers['18'] = '925. K-19: The Widowmaker (tt0267626)' # +

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [49]:
data.groupby(['release_year'])[['revenue']].sum().sort_values(['revenue'], ascending=False).head(1)

,revenue
release_year,
2015,25449202382


In [50]:
answers['19'] = '2015' # +

# 20. Какой самый прибыльный год для студии Warner Bros?

In [51]:
# Создаем Датафрейм с фильмами, выпущенными компанией Warner Brosers
data_WB = data_prod[data_prod['production_companies_list'].str.contains("Warner")]
data_WB.groupby(['release_year'])[['profit']].sum().sort_values(['profit'], ascending=False).head(1)

,profit
release_year,
2014,2292949646


In [52]:
answers['20'] = '2014' # +

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [53]:
data_time.groupby(['release_month'])[['original_title']].count().sort_values(['original_title'], ascending=False).head(1)

,original_title
release_month,
9,227


In [54]:
answers['21'] = '9' # +

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [55]:
data_time[(data_time.release_month == '6')|(data_time.release_month == '7')|(data_time.release_month == '8')]['original_title'].count()

450

In [56]:
answers['22'] = '450' # +

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [57]:
data_winter_temp = data_time.explode('director_list')
data_winter = data_winter_temp[(data_winter_temp.release_month == '1')|(data_winter_temp.release_month == '2')|(data_winter_temp.release_month == '12')]
data_winter.groupby(['director_list'])[['original_title']].count().sort_values(['original_title'], ascending=False).head(1)

,original_title
director_list,
Peter Jackson,7


In [58]:
answers['23'] = 'Peter Jackson' # +

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [59]:
data_lenght = data.explode('production_companies_list')
data_lenght.groupby(['production_companies_list'])[['lenght_title']].mean().sort_values(['lenght_title'], ascending=False).head(1)

,lenght_title
production_companies_list,
Four By Two Productions,83.0


In [60]:
answers['24'] = 'Four By Two Productions' # +

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [61]:
data_lenght_overview = data.explode('production_companies_list')
data_lenght_overview.groupby(['production_companies_list'])[['lenght_overview']].mean().sort_values(['lenght_overview'], ascending=False).head(1)

,lenght_overview
production_companies_list,
Midnight Picture Show,175.0


In [62]:
answers['25'] = 'Midnight Picture Show' # +

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [63]:
border = np.quantile(data.vote_average, 0.99)
best_films = data[data.vote_average >= border].original_title
best_films_list = []
for film in best_films:
    best_films_list.append(film)
answers['26'] = best_films_list
print(answers['26'])

['Inside Out', 'Spotlight', 'Room', 'Interstellar', 'Guardians of the Galaxy', 'Big Hero 6', 'The Imitation Game', 'Gone Girl', 'The Grand Budapest Hotel', 'The Theory of Everything', 'The Fault in Our Stars', 'Mr. Nobody', '3 Idiots', 'Inception', 'The Lord of the Rings: The Fellowship of the Ring', 'The Dark Knight', 'The Lord of the Rings: The Two Towers', 'The Pianist', 'The Lord of the Rings: The Return of the King', 'The Wolf of Wall Street', 'Her', '12 Years a Slave', 'Prisoners', 'Dallas Buyers Club', 'The Prestige', 'Eternal Sunshine of the Spotless Mind', 'There Will Be Blood', 'Memento']


In [64]:
answers['26'] = [
    'Inside Out', 'Spotlight', 'Room', 'Interstellar', 'Guardians of the Galaxy', 'Big Hero 6',
    'The Imitation Game', 'Gone Girl', 'The Grand Budapest Hotel', 'The Theory of Everything',
    'The Fault in Our Stars', 'Mr. Nobody', '3 Idiots', 'Inception',
    'The Lord of the Rings: The Fellowship of the Ring', 'The Dark Knight',
    'The Lord of the Rings: The Two Towers', 'The Pianist', 'The Lord of the Rings: The Return of the King',
    'The Wolf of Wall Street', 'Her', '12 Years a Slave', 'Prisoners', 'Dallas Buyers Club', 'The Prestige',
    'Eternal Sunshine of the Spotless Mind', 'There Will Be Blood', 'Memento'
] # +

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [65]:
data_pairs = data_p.explode('pairs')
collections.Counter(data_pairs['pairs']).most_common(1)

[(('Daniel Radcliffe', 'Rupert Grint'), 8)]

In [66]:
answers['27'] = 'Daniel Radcliffe', 'Rupert Grint' # +

# Submission

In [67]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '723. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '1157. Gods and Generals (tt0279111)',
 '3': '768. Winnie the Pooh (tt1449283)',
 '4': '110',
 '5': '107.0',
 '6': '239. Avatar (tt0499549)',
 '7': '1245. The Lone Ranger (tt1210819)',
 '8': '1478',
 '9': '599. The Dark Knight (tt0468569)',
 '10': '1245. The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': '925. K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': '9',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': ['Inside Out',
  'Spotlight',
  'Room',
  'Interstellar',
  'Guardians of the Galaxy',
  'Big Hero 6',
  'The Imitation Game',
  'Gone Girl',
  'The Grand Budapest Hotel',
  'The Theory of Everything',
  'The Fault in Our Stars',
  'Mr. Nobody',
  '3 Idiots',
  'Inception',
  'The Lord of t

In [68]:
# и убедиться что ни чего не пропустил)
len(answers)

27